# egoplan preprocess

In [1]:

import json
egoplan_annotations = json.load(open('./EgoPlan-Bench2.json'))
ego4d_metadata = json.load(open('/mnt/extra/dataset/ego4d/ego4d.json'))['videos']

egoplan_annotations_processed = []
for sample in egoplan_annotations:
    video_id = sample['sample_id'].split('_')[0]
    for video in ego4d_metadata:
        if video_id == video['video_uid']:
            break
    fps = video['video_metadata']['fps']
    sample['fps'] = fps
    sample['video_uid'] = video_id
    egoplan_annotations_processed.append(sample)

json.dump(egoplan_annotations_processed, open('./EgoPlan-Bench2-processed.json', 'w'), indent=4)



In [3]:
import json
egoplan_annotations_processed = json.load(open('./EgoPlan-Bench2-processed.json'))

In [4]:
import random
print(random.random())

0.3632605018306919


## transform

In [5]:

import random
import numpy as np

instructions_mode = [
    {"role": "user", "content": "Can you guide me step by step on how to achieve my goal, such as {}?"},
    {"role": "user", "content": "What are the steps I need to follow to successfully accomplish my goal, like {}?"},
    {"role": "user", "content": "Could you break down the process of achieving my goal, such as {}, into simple steps?"},
    {"role": "user", "content": "What should I do first, second, and so on to accomplish my goal, like {}?"},
    {"role": "user", "content": "Please provide me with a detailed step-by-step guide to achieving my goal, such as {}."},
    {"role": "user", "content": "How can I achieve my goal, such as {}, step by step? Can you explain each stage?"},
    {"role": "user", "content": "Can you walk me through the entire process of achieving my goal, such as {}, one step at a time?"},
    {"role": "user", "content": "What are the sequential actions I need to take to accomplish my goal, like {}?"},
    {"role": "user", "content": "Could you assist me by outlining the step-by-step instructions for achieving my goal, such as {}?"},
    {"role": "user", "content": "Please explain how to achieve my goal {}, starting from the very beginning and detailing each step."},
]

next_instructions = [
    {"role": "user", "content": "What are the next steps to {} after this? Please explain in detail."},
    {"role": "user", "content": "Can you guide me on what to do next to {}, including specific actions?"},
    {"role": "user", "content": "What should I do next to continue doing {}? Please provide clear instructions."},
    {"role": "user", "content": "Could you explain the following steps for {} with all necessary details?"},
    {"role": "user", "content": "What comes after this step in the process of {}? Please describe thoroughly."},
    {"role": "user", "content": "Please tell me the next actions required to {}, step by step."},
    {"role": "user", "content": "What is the next thing I need to do to {}? Include any important details."},
    {"role": "user", "content": "Can you outline the subsequent steps for bread preparation with detailed guidance?"},
    {"role": "user", "content": "What should be done next to complete the process of {}? Please elaborate."},
    {"role": "user", "content": "What are the following steps for {} successfully? Provide specific actions and details."},
]


def sample2planqa(sample, mode_ratio=0.5, new_fps=2):
    
    fps = sample['fps']
    conversation = {
        'video_uid': sample['sample_id'].split('_')[0],
        'conversation': [],
        'duration': sample['current_observation_frame'] / fps - sample['task_start_frame'] / fps
    }
    
    if random.random() < mode_ratio: # instruction mode
        
        time = sample['task_start_frame'] / fps
        wait_time = min(time, 1/new_fps)
        start_time = time - wait_time
        
        conversation['conversation'].append({
            'role': 'user',
            'content': random.choice(instructions_mode)['content'].format(sample['task_goal']),
            'time': start_time,
        }) # user asks for instructions
        
        for action in sample['task_progress_metadata']: # generate assistant's response
            conversation['conversation'].append({
            'role': 'assistant',
            'content': 'You need to {}.'.format(action['narration_text']),
            'time': time,
            })
            time = action['stop_frame'] / fps
        
        conversation['conversation'].append({
            'role': 'assistant',
            'content': 'You need to {}'.format(sample['answer']),
            'time': time,
        })
        
    else:
        # select a random action, then ask for next steps
        action_number = len(sample['task_progress_metadata'])
        insert_index = random.randint(0, action_number-1-1) # insert the question before the last action
        time = sample['task_progress_metadata'][insert_index+1]['start_frame'] / fps
        last_time = sample['task_progress_metadata'][insert_index]['stop_frame'] / fps 
        wait_time = random.random() * (time - last_time)
        start_time = time - wait_time
        
        conversation['conversation'].append({
            'role': 'user',
            'content': random.choice(next_instructions)['content'].format(sample['task_goal']),
            'time': start_time,
        })
        
        # recored the previous actions
        preview_actions = []
        for i, action in enumerate(sample['task_progress_metadata']):
            if i <= insert_index:
                preview_actions.append(action['narration_text'])
        
        conversation['conversation'].append({
            'role': 'assistant',
            'content': 'You have done {}.'.format(', '.join(preview_actions)) + ' Next, you need to {}.'.format(sample['task_progress_metadata'][insert_index+1]['narration_text']),
            'time': time,
        })
        time = sample['task_progress_metadata'][insert_index+1]['stop_frame'] / fps
        
        # recored the following actions
        for action in sample['task_progress_metadata'][insert_index+2:]:
            conversation['conversation'].append({
            'role': 'assistant',
            'content': 'You need to {}.'.format(action['narration_text']),
            'time': time,
            })
            time = action['stop_frame'] / fps
        
        # recored the last action
        conversation['conversation'].append({
            'role': 'assistant',
            'content': 'You need to {}'.format(sample['answer']),
            'time': time,
        })
    
    return conversation

print(json.dumps(sample2planqa(egoplan_annotations_processed[0], 0.5), indent=4))

{
    "video_id": "0d270946-95c5-4e71-ae49-b9e802548147",
    "conversation": [
        {
            "role": "user",
            "content": "What are the sequential actions I need to take to accomplish my goal, like prepare bread?",
            "time": 778.9333333333333
        },
        {
            "role": "assistant",
            "content": "You need to scoop dough.",
            "time": 779.4333333333333
        },
        {
            "role": "assistant",
            "content": "You need to scoop dough.",
            "time": 780.7
        },
        {
            "role": "assistant",
            "content": "You need to knead dough.",
            "time": 781.8
        },
        {
            "role": "assistant",
            "content": "You need to place dough on plate.",
            "time": 785.0333333333333
        },
        {
            "role": "assistant",
            "content": "You need to add oil.",
            "time": 789.6333333333333
        },
        {
           

# goal step

In [3]:
import json, os

def get_narrations(sources):
    annos = []
    for source in sources:
        if source['segments']:
            annos.append({
                'video_uid': source['video_uid'],
                'summary': (source['start_time'], source['end_time'], source['goal_description'].strip()),
                'narrations': [(segment['start_time'], segment['end_time'], segment['step_description'].strip()) for segment in source['segments']],
            })
        for segment in source['segments']:
            if segment['segments']:
                annos.append({
                    'video_uid': source['video_uid'],
                    'summary': (segment['start_time'], segment['end_time'], segment['step_description'].strip()),
                    'narrations': [(seg['start_time'], seg['end_time'], seg['step_description'].strip()) for seg in segment['segments']],
                })
                if segment['start_time'] > segment['segments'][0]['start_time'] or segment['end_time'] < segment['segments'][-1]['end_time']:
                    print(source['video_uid'], segment['start_time'], segment['end_time'], segment['segments'][0]['start_time'], segment['segments'][-1]['end_time'])
    return annos
EGO4D_ANNO_ROOT = '/mnt/extra/dataset/ego4d/v2/annotations/'
sources = json.load(open(os.path.join(EGO4D_ANNO_ROOT,'goalstep_train.json')))['videos']
annos = get_narrations(sources)
sources = json.load(open(os.path.join(EGO4D_ANNO_ROOT,'goalstep_val.json')))['videos']
annos = annos + get_narrations(sources)
print(len(annos))

grp-090c6bc0-49da-4d3b-b209-a1a60aeb0317 113.56319 186.37196000000003 119.82558 186.73328
grp-090c6bc0-49da-4d3b-b209-a1a60aeb0317 1167.8622149999999 1199.435 1167.44849 1199.41459
grp-090c6bc0-49da-4d3b-b209-a1a60aeb0317 1627.61174 1642.5281100000002 1627.63749 1642.60212
grp-090c6bc0-49da-4d3b-b209-a1a60aeb0317 1760.45286 1785.1901900000003 1761.47556 1785.79746
4c642620-db0e-4096-9ece-2b2c6fdb47b0 0 62.64435 0.39335 63.626
4c642620-db0e-4096-9ece-2b2c6fdb47b0 2324.2209 2413.00206 2335.8445 2413.21675
4c642620-db0e-4096-9ece-2b2c6fdb47b0 3794.07976 3893.2037 3796.12032 3893.5152
4c642620-db0e-4096-9ece-2b2c6fdb47b0 4146.67089 4220.54328 4147.89852 4220.77077
grp-ab070b36-def2-4ad7-a760-2a9ce29ce505 445.92881 474.54501 445.94881 474.73336
grp-d250521e-5197-44aa-8baa-2f42b24444d2 269.65659 586.29644 284.79803 586.42983
546a1aed-676d-44ed-a63c-8db89fa4d935 200.7685 230.68778 200.45397 229.49702
94d5eff8-0fac-4719-adf2-5c0208ab89f7 437.44151 618.485 437.441 618.45829
91e3e6ce-bc01-4720-a

In [4]:
goalstep_annotation_processed = []
for anno in annos:
    processed_anno = {
        'sample_id': anno['video_uid'],
        'task_goal': anno['summary'][2],
        'task_start_frame': anno['summary'][0], # it is time
        'current_observation_frame': anno['summary'][1], # it is time
    }
    processed_anno['task_progress_metadata'] = []
    for nar in anno['narrations']:
        processed_anno['task_progress_metadata'].append({
            'start_frame': nar[0],
            'stop_frame': nar[1],
            'narration_text': nar[2],
        })
    goalstep_annotation_processed.append(processed_anno)

json.dump(goalstep_annotation_processed, open('./goalstep_annotation_processed.json', 'w'), indent=4)

In [5]:
import json
annos = json.load(open('/home/zhangyl/videollm-online/data/estp/egoplan/goalstep_annotation_processed.json'))
for anno in annos:
    if anno['current_observation_frame'] < anno['task_progress_metadata'][-1]['stop_frame']:
        print(anno['sample_id'])
        print(anno['current_observation_frame'], anno['task_progress_metadata'][-1]['stop_frame'])
        break
    if anno['task_start_frame'] > anno['task_progress_metadata'][0]['start_frame']:
        print(anno['sample_id'])
        break

    

grp-090c6bc0-49da-4d3b-b209-a1a60aeb0317
186.37196000000003 186.73328


In [6]:


import random
import numpy as np

instructions_mode = [
    {"role": "user", "content": "Can you guide me step by step on how to achieve my goal, such as {}?"},
    {"role": "user", "content": "What are the steps I need to follow to successfully accomplish my goal, like {}?"},
    {"role": "user", "content": "Could you break down the process of achieving my goal, such as {}, into simple steps?"},
    {"role": "user", "content": "What should I do first, second, and so on to accomplish my goal, like {}?"},
    {"role": "user", "content": "Please provide me with a detailed step-by-step guide to achieving my goal, such as {}."},
    {"role": "user", "content": "How can I achieve my goal, such as {}, step by step? Can you explain each stage?"},
    {"role": "user", "content": "Can you walk me through the entire process of achieving my goal, such as {}, one step at a time?"},
    {"role": "user", "content": "What are the sequential actions I need to take to accomplish my goal, like {}?"},
    {"role": "user", "content": "Could you assist me by outlining the step-by-step instructions for achieving my goal, such as {}?"},
    {"role": "user", "content": "Please explain how to achieve my goal {}, starting from the very beginning and detailing each step."},
]

next_instructions = [
    {"role": "user", "content": "What are the next steps to {} after completing the steps of {}? Please explain in detail."},
    {"role": "user", "content": "Can you guide me on what to do next to {}, considering the steps I've already completed: {}? Include specific actions."},
    {"role": "user", "content": "What should I do next to continue {} after completing the earlier tasks: {}? Please provide clear instructions."},
    {"role": "user", "content": "Could you explain the following steps for {} based on the steps I have already completed: {}? Provide all necessary details."},
    {"role": "user", "content": "What comes after this step in the process of {} after completing the steps: {}? Please describe thoroughly."},
    {"role": "user", "content": "Please tell me the next actions required to {}, step by step, considering the steps I've already completed: {}."},
    {"role": "user", "content": "What is the next thing I need to do to {} after the steps already completed: {}? Include any important details."},
    {"role": "user", "content": "Can you outline the subsequent steps for {} after completing the previous steps: {}? Provide detailed guidance."},
    {"role": "user", "content": "What should be done next to complete the process of {} after finishing the earlier steps: {}? Please elaborate."},
    {"role": "user", "content": "What are the following steps for {} successfully, after the completion of the steps: {}? Provide specific actions and details."},
]


def sample2planqa(sample, mode_ratio=0.5, new_fps=2):
    
    fps = 1
    conversation = {
        'video_uid': sample['sample_id'].split('_')[0],
        'conversation': [],
        'duration': sample['current_observation_frame'] / fps - sample['task_start_frame'] / fps,
        'start_time': sample['task_start_frame'] / fps,
        'end_time': sample['current_observation_frame'] / fps,
    }
    
    if random.random() < mode_ratio: # instruction mode
        
        # first turn
        start_time = sample['task_start_frame'] / fps
        wait_time = min(start_time, 1/new_fps) # insert question in the first 1/new_fps seconds
        question_time = start_time - wait_time
        end_time = sample['task_progress_metadata'][0]['start_frame'] / fps
        
        # construct the conversation
        fps = 1
        conversation = {
            'video_uid': sample['sample_id'].split('_')[0],
            'conversation': [],
            'duration': sample['current_observation_frame'] / fps -  question_time,
            'start_time': question_time,
            'end_time': sample['current_observation_frame'] / fps,
            'Task Type': 'Task Understanding',
        }
        
        conversation['conversation'].append({
            'role': 'user',
            'content': random.choice(instructions_mode)['content'].format(sample['task_goal']),
            'time': question_time,
        }) # user asks for instructions
        
        for action in sample['task_progress_metadata']: # generate assistant's response
            conversation['conversation'].append({
            'role': 'assistant',
            'content': 'You need to {}.'.format(action['narration_text']),
            'time': end_time,
            'start_time': start_time,
            'end_time': end_time,
            'stamp_time': start_time + (end_time - start_time) / 2,
            })
            start_time = action['start_frame'] / fps
            end_time = action['stop_frame'] / fps
        
        # conversation['conversation'].append({
        #     'role': 'assistant',
        #     'content': 'You need to {}'.format(sample['answer']),
        #     'time': time,
        # })
        
    else:
        
        # select a random action, then ask for next steps
        action_number = len(sample['task_progress_metadata'])
        insert_index = random.randint(0, action_number-1-1) # insert the question before the last action
        
        
        # first turn
        end_time = sample['task_progress_metadata'][insert_index+1]['start_frame'] / fps
        start_time = sample['task_progress_metadata'][insert_index]['stop_frame'] / fps 
        question_time = start_time
        
        fps = 1
        conversation = {
            'video_uid': sample['sample_id'].split('_')[0],
            'conversation': [],
            'duration': sample['current_observation_frame'] / fps - question_time,
            'start_time': question_time,
            'end_time': sample['current_observation_frame'] / fps,
            'Task Type': 'Action Reasoning',
        }
        
        
        # insert the question
        preview_actions = []
        for i, action in enumerate(sample['task_progress_metadata']):
            if i <= insert_index:
                preview_actions.append(action['narration_text'])
                
        conversation['conversation'].append({
            'role': 'user',
            'content': random.choice(next_instructions)['content'].format(sample['task_goal'], ', '.join(preview_actions).lower()),
            'time': question_time,
        })
        
        # recored the following actions
        for action in sample['task_progress_metadata'][insert_index+1:]:
            conversation['conversation'].append({
            'role': 'assistant',
            'content': 'You need to {}.'.format(action['narration_text']),
            'time': end_time,
            'start_time': start_time,
            'end_time': end_time,
            'stamp_time': start_time + (end_time - start_time) / 2,
            })
            start_time = action['start_frame'] / fps
            end_time = action['stop_frame'] / fps
        
        # recored the last action
        # conversation['conversation'].append({
        #     'role': 'assistant',
        #     'content': 'You need to {}'.format(sample['answer']),
        #     'time': time,
        # })
    
    return conversation

print(json.dumps(sample2planqa(goalstep_annotation_processed[1], 0.5), indent=4))

{
    "video_uid": "39d087b0-afc2-47d8-ba91-b70dd8fab90e",
    "conversation": [
        {
            "role": "user",
            "content": "What are the steps I need to follow to successfully accomplish my goal, like preheat pots?",
            "time": 1068.848
        },
        {
            "role": "assistant",
            "content": "You need to Add wood to fire.",
            "time": 1070.09237,
            "start_time": 1069.348,
            "end_time": 1070.09237,
            "stamp_time": 1069.7201850000001
        },
        {
            "role": "assistant",
            "content": "You need to Set stands on fireplace.",
            "time": 1158.54893,
            "start_time": 1070.09237,
            "end_time": 1158.54893,
            "stamp_time": 1114.3206500000001
        },
        {
            "role": "assistant",
            "content": "You need to Boil water.",
            "time": 1189.912,
            "start_time": 1158.78141,
            "end_time": 1189.912,
  

## refine

In [38]:
print(len(goalstep_annotation_processed))

5647


In [ ]:

import json
from openai import OpenAI
from ast import literal_eval

# def get_llm_reponse_json(system_prompt, user_prompt):
#     client = OpenAI(
#             api_key="",
#             base_url="https://api.deepseek.com",
#         )

#     messages = [{"role": "system", "content": system_prompt},
#                 {"role": "user", "content": user_prompt}]

#     response = client.chat.completions.create(
#         model="deepseek-chat",
#         messages=messages,
#         response_format={
#             'type': 'json_object'
#         }
#     )
#     return response.choices[0].message.content

def get_llm_reponse_json(system_prompt, user_prompt):
    client = OpenAI(
            api_key='',
            base_url="https://ark.cn-beijing.volces.com/api/v3",
        )

    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content":  user_prompt}]

    response = client.chat.completions.create(
        model="ep-20250209171240-27r8g",
        messages=messages,
        response_format={
            'type': 'json_object'
        }
    )
    return response.choices[0].message.content

system_prompt = open('./refine_system_prompt.txt').read()
output_file = './goalstep_annotation_processed_explict_v3.json'
repeat_times = 5

if os.path.exists(output_file):
    goalstep_annotation_processed_explict = json.load(open(output_file))
    last_index = 0
else:
    goalstep_annotation_processed_explict = []
    last_index = 0
    

for i in range(last_index, len(goalstep_annotation_processed)):
    if i % 10 == 0:
        json.dump(goalstep_annotation_processed_explict, open(output_file, 'w'), indent=4)
        print(i)
        
    for _ in range(repeat_times):
        if len(goalstep_annotation_processed[i]['task_progress_metadata']) < 2:
            continue
        # try:
        user_prompt = sample2planqa(goalstep_annotation_processed[i], 0.5)
        duration = user_prompt['duration']
        if duration < 60 or duration > 3600:
            continue
        try:
            # raw_response = get_llm_reponse_json(system_prompt, json.dumps(user_prompt, indent=4))
            # raw_response = raw_response.replace('```json', '').replace('```', '')
            # refine_conve = json.loads(raw_response)
            raw_response = get_llm_reponse_json(system_prompt, json.dumps(user_prompt, indent=4))
            refine_conve = json.loads(raw_response)
        except:
            try:
                refine_conve = literal_eval(str(raw_response))
            except:
                print(i)
                continue
        for ori_text, refine_text in zip(user_prompt['conversation'], refine_conve['conversation']):
            if ori_text['role'] == refine_text['role']:
                ori_text['content'] = refine_text['content']
        
        
        goalstep_annotation_processed_explict.append(user_prompt)
    
    # except:
    #     print(i)
    #     continue


json.dump(goalstep_annotation_processed_explict, open(output_file, 'w'), indent=4)


0
10
20
30
30
30
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
313
313
313
320
330
340
340
350
360
370
380
390
400
410
420
430
440
450
460
468
468
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
795
800
800
800
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1013
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1208
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1421
1430
1436
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1712
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1877
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
20

In [1]:
# calibration start_time and end_time
import json, os
data = json.load(open('/home/zhangyl/videollm-online/data/estp/egoplan/goalstep_annotation_processed_explict_v3.json'))
for d in data:
    if d['start_time'] > d['conversation'][0]['time']:
        d['start_time'] = d['conversation'][0]['time']
        
    if d['end_time'] < d['conversation'][-1]['time']:
        d['end_time'] = d['conversation'][-1]['time']

json.dump(data, open('/home/zhangyl/videollm-online/data/estp/egoplan/goalstep_annotation_processed_explict_v3_calibrated.json', 'w'), indent=4)


In [2]:
def time_consistent(conversation):
    
    last_start_time = -1
    last_end_time = -1
    
    for c in conversation:
        if 'start_time' in c:
            if c['start_time'] > c['end_time']:
                return False
            if c['start_time'] < last_start_time:
                return False
            if c['start_time'] < last_end_time:
                return False
            last_start_time = c['start_time']
            last_end_time = c['end_time']
    return True
            
            

In [4]:
import json, os
data = json.load(open('/home/zhangyl/videollm-online/data/estp/egoplan/goalstep_annotation_processed_explict_v3_calibrated.json'))
clear_data = []
print(len(data))
for d in data:
    if time_consistent(d['conversation']):
        clear_data.append(d)
print(len(clear_data))
json.dump(clear_data, open('/home/zhangyl/videollm-online/data/estp/annotation_train/0/goalstep_annotation_processed_explict_v3_calibrated_consistent.json', 'w'), indent=4)


12310
10839


# train data

In [5]:
import json, os
EGO4D_ANNO_ROOT = '/mnt/extra/dataset/ego4d/v2/annotations/'
data = json.load(open('/home/zhangyl/videollm-online/data/estp/egoplan/goalstep_annotation_processed_explict_v3_calibrated_consistent.json'))
valid_data = []
print(len(data))

valid_video_id = []
valid_sources = json.load(open(os.path.join(EGO4D_ANNO_ROOT,'goalstep_train.json')))['videos']
for source in valid_sources:
    valid_video_id.append(source['video_uid'])

for d in data:
    if d['video_uid'] in valid_video_id:
        valid_data.append(d)

print(len(valid_data))

json.dump(valid_data, open('/home/zhangyl/videollm-online/data/estp/annotation_train/0/goalstep_annotation_processed_explict_v3_calibrated_consistent_train.json', 'w'), indent=4)


10839
8782


# valid data

In [11]:
import json, os
EGO4D_ANNO_ROOT = '/mnt/extra/dataset/ego4d/v2/annotations/'
data = json.load(open('/home/zhangyl/videollm-online/data/estp/egoplan/goalstep_annotation_processed_explict_v2_calibrated_consistent.json'))
valid_data = []
print(len(data))

valid_video_id = []
valid_sources = json.load(open(os.path.join(EGO4D_ANNO_ROOT,'goalstep_val.json')))['videos']
for source in valid_sources:
    valid_video_id.append(source['video_uid'])

for d in data:
    if d['video_uid'] in valid_video_id:
        valid_data.append(d)

print(len(valid_data))

# Filter data for Action Reasoning and Task Understanding tasks with duration < 600
action_reasoning_samples = []
task_understanding_samples = []

for sample in valid_data:
    task_type = sample.get('Task Type')
    # Calculate duration from clip start and end times
    duration = sample.get('end_time', 0) - sample.get('start_time', 0)
    
    # Only include samples with duration less than 600 seconds
    if duration < 1000 and time_consistent(sample['conversation']):
        if task_type == 'Action Reasoning':
            action_reasoning_samples.append(sample)
        elif task_type == 'Task Understanding':
            task_understanding_samples.append(sample)

# Randomly select 100 samples from each task type (or all if less than 100)
import random
selected_action_reasoning = random.sample(action_reasoning_samples, min(100, len(action_reasoning_samples)))
selected_task_understanding = random.sample(task_understanding_samples, min(100, len(task_understanding_samples)))

# Print statistics
print(f"Total Action Reasoning samples (duration < 600s): {len(action_reasoning_samples)}")
print(f"Selected Action Reasoning samples: {len(selected_action_reasoning)}")
print(f"Total Task Understanding samples (duration < 600s): {len(task_understanding_samples)}")
print(f"Selected Task Understanding samples: {len(selected_task_understanding)}")

# Combine the selected samples
selected_samples = selected_action_reasoning + selected_task_understanding
print(f"Total selected samples: {len(selected_samples)}")

selected_samples_dict = {}
for sample in selected_samples:
    if sample['video_uid'] not in selected_samples_dict:
        selected_samples_dict[sample['video_uid']] = {"goalstep": []}
    selected_samples_dict[sample['video_uid']]['goalstep'].append(sample)

# json.dump(selected_samples_dict, open('/home/zhangyl/videollm-online/data/estp/egoplan/selected_action_task_samples.json', 'w'), indent=4)



2199
417
Total Action Reasoning samples (duration < 600s): 159
Selected Action Reasoning samples: 100
Total Task Understanding samples (duration < 600s): 221
Selected Task Understanding samples: 100
Total selected samples: 200


In [86]:
def print_json(json_data):
    print(json.dumps(json_data, indent=4))
    
    
def count_task_types(refine_annos):
    task2number = {
        "Object State Change Recognition": 0,
        "Ego Object State Change Recognition": 0,
        "Object Localization": 0,
        "Action Recognition": 0,
        "Action Reasoning": 0,
        "Object Recognition": 0,
        "Ego Object Localization": 0,
        "Object Function": 0,
        "Task Understanding": 0,
        "Attribute Perception": 0,
        "Information Function": 0,
        "Text-Rich Understanding": 0
    }
    c = 0
    c_1 = 0
    c_2 = 0
    for k,v in refine_annos.items():
        c_2 += 1
        for kk,vv in v.items():
            c_1 += 1
            for ll in vv:
                task2number[ll['Task Type'].strip()] += 1
                c += 1
    print_json(task2number)
    print(c)
    print(c_1)
    print(c_2)
    return task2number  


def merge_dict(dict1, dict2):
    for k,v in dict2.items():
        if k in dict1:
            dict1[k].update(v)
        else:
            dict1[k] = v
    return dict1

import json

data = json.load(open("/home/zhangyl/videollm-online/refine_annos_last.json"))
merged_data = merge_dict(data, selected_samples_dict)


for k,v in merged_data.items():
    for kk,vv in v.items():
        for qa in vv:
            if len(qa['conversation']) == 0:
                print(k,kk)
            if 'question' not in qa:
                print(k,kk)


json.dump(merged_data, open('/home/zhangyl/videollm-online/estp_bench_sq.json', 'w'), indent=4)
video_list = list(merged_data.keys())
with open('/home/zhangyl/videollm-online/estp_bench_sq_video_list.txt', 'w') as f:
    for video in video_list:
        f.write(video + '\n')

count_task_types(merged_data)


NameError: name 'selected_samples_dict' is not defined

In [ ]:
import json
annos = json.load(open('/home/zhangyl/videollm-online/data/estp/egoplan/goalstep_annotation_processed.json'))
for anno in annos:
    if anno['current_observation_frame'] < anno['task_progress_metadata'][-1]['stop_frame']:
        print(anno['sample_id'])
        print(anno['current_observation_frame'], anno['task_progress_metadata'][-1]['stop_frame'])
        break
    if anno['task_start_frame'] > anno['task_progress_metadata'][0]['start_frame']:
        print(anno['sample_id'])
        break

    

grp-090c6bc0-49da-4d3b-b209-a1a60aeb0317
186.37196000000003 186.73328


# Contextual Goalstep

In [2]:
import json,os,re

anno_dir = '/home/zhangyl/videollm-online/datasets/ego4d/v2/annotations/livechat_deepseek/'
print(len(os.listdir(anno_dir)))

annos = []

for anno_file in os.listdir(anno_dir):
    anno = json.load(open(os.path.join(anno_dir, anno_file)))
    annos.append(anno)

new_annos = []
for anno in annos:
    if not anno['conversation']:
        continue
    maintain = True
    anno['duration'] = anno['conversation'][-1]['time'] - anno['conversation'][0]['time']
    if anno['duration'] < 60 or anno['duration'] > 1800:
        continue
    for message in anno['conversation']:
        if 'second' in message['content'] or re.match(r'\b\d+s\b', message['content']): # if the generated content contains time related text, it may leak the future ground-truth 
            maintain = False
            break
    if maintain:
        new_annos.append(anno)

print(len(new_annos))




2195
1064


In [3]:
def time_consistent(conversation):
    
    last_start_time = -1
    last_end_time = -1
    # Sort conversation by start_time if present
    sorted_conversation = sorted([c for c in conversation if 'start_time' in c], key=lambda x: x['start_time'])
    unsorted_conversation = [c for c in conversation if 'start_time' not in c]
    conversation = unsorted_conversation + sorted_conversation
    for c in sorted_conversation:
        if 'start_time' in c:
            if c['start_time'] > c['end_time']:
                return False
            if c['start_time'] < last_start_time:
                return False
            if c['start_time'] < last_end_time:
                return False
            last_start_time = c['start_time']
            last_end_time = c['end_time']
    return True

def text_consistent(conversation):
    for c in conversation:
        if 'content' in c:
            if c['content'] == '...':
                return False
    return True

def clear_text(conversation):
    return [c for c in conversation if not ('content' in c and c['content'] == '...')]


def clear_user_text(conversation):
    """
    Remove user messages that are not followed by assistant messages.
    
    Args:
        conversation: List of conversation messages
        
    Returns:
        List of conversation messages with orphaned user messages removed
    """
    filtered_conversation = []
    i = 0
    
    while i < len(conversation):
        if conversation[i]['role'] == 'User':
            # Check if there's at least one assistant response after this user query
            if i + 1 < len(conversation) and conversation[i + 1]['role'] == 'Assistant':
                # Add this user message and continue processing
                filtered_conversation.append(conversation[i])
            # If no assistant response follows, skip this user message
        else:
            # Add all assistant messages
            filtered_conversation.append(conversation[i])
        i += 1
    
    return filtered_conversation

def conversation_consistent(conversation):
    i = 0
    while i < len(conversation):
        if conversation[i]['role'] == 'User':
            # Check if next message exists and is from Assistant
            if i + 1 >= len(conversation) or conversation[i+1]['role'] != 'Assistant':
                return False
        i += 1
    return True

def user_number_consistent(conversation):
    user_number = 0
    for c in conversation:
        if c['role'] == 'User':
            user_number += 1
    if user_number < 2:
        return False
    return True


In [5]:
import json
valid_video_id = []
valid_data = []
time_not_consistent = []
mode = 'val'
EGO4D_ANNO_ROOT = '/home/zhangyl/videollm-online/datasets/ego4d/v2/annotations'
valid_sources = json.load(open(os.path.join(EGO4D_ANNO_ROOT,'goalstep_{}.json'.format(mode))))['videos']
for source in valid_sources:
    valid_video_id.append(source['video_uid'])


c = 0   
for d in new_annos:
    # d['conversation'] = clear_text(d['conversation'])
    # d['conversation'] = clear_user_text(d['conversation'])
    if not user_number_consistent(d['conversation']):
        continue
    c += 1
    if not time_consistent(d['conversation']):
        time_not_consistent.append(d)
    if d['video_uid'] in valid_video_id and time_consistent(d['conversation']) and text_consistent(d['conversation']) and user_number_consistent(d['conversation']):
        valid_data.append(d)

print(c)
print(len(time_not_consistent))
print(len(valid_data))
json.dump(valid_data, open('/home/zhangyl/videollm-online/data/estp/annotation_train/0/goalstep_{}_valid.json'.format(mode), 'w'), indent=4)

949
70
364


In [17]:
# Select 200 samples from valid_data with as many different video_uids as possible
import random

def select_diverse_samples(data, num_samples=231):
    # Get all unique video_uids
    video_uids = set(item['video_uid'] for item in data)
    print(f"Total unique video_uids: {len(video_uids)}")
    
    # Create a dictionary to store samples by video_uid
    video_to_samples = {}
    for item in data:
        video_uid = item['video_uid']
        if video_uid not in video_to_samples:
            video_to_samples[video_uid] = []
        video_to_samples[video_uid].append(item)
    
    selected_samples = []
    
    # First, try to select one sample from each unique video_uid
    for video_uid in video_uids:
        if len(selected_samples) >= num_samples:
            break
        samples = video_to_samples[video_uid]
        selected_samples.append(random.choice(samples))
    
    # If we still need more samples, randomly select from remaining videos
    remaining_samples = [item for item in data if item not in selected_samples]
    if len(selected_samples) < num_samples and remaining_samples:
        additional_needed = min(num_samples - len(selected_samples), len(remaining_samples))
        selected_samples.extend(random.sample(remaining_samples, additional_needed))
    
    print(f"Selected {len(selected_samples)} samples")
    
    # Count unique video_uids in the selected samples
    selected_video_uids = set(item['video_uid'] for item in selected_samples)
    print(f"Number of unique video_uids in selected samples: {len(selected_video_uids)}")
    
    return selected_samples

# Select diverse samples
fileter_data = []
for d in valid_data:
    if d["clip_end_time"] - d["clip_start_time"] < 10 or d["clip_end_time"] - d["clip_start_time"] > 1300:
        continue
    fileter_data.append(d)
print(len(fileter_data))
selected_samples = select_diverse_samples(fileter_data, num_samples=211)


print(len(selected_samples))
with open('/home/zhangyl/videollm-online/data/estp/annotation_train/0/goalstep_val_valid_samples_v3.json', 'w') as f:
    json.dump(selected_samples, f, indent=4)

309
Total unique video_uids: 87
Selected 211 samples
Number of unique video_uids in selected samples: 87
211


In [18]:
# 打印第一个不满足time_consistent的对话
print("First conversation that fails time consistency check:")
# print(json.dumps(time_not_consistent[0], indent=4))

# 保存到文件以便查看
with open('./temp.txt', 'w') as f:
    f.write(json.dumps(time_not_consistent[0], indent=4))

# 分析不满足time_consistent的原因
print("\nAnalyzing time inconsistency issues:")
for i, msg in enumerate(time_not_consistent[0]['conversation']):
    if i > 0:
        prev_msg = time_not_consistent[0]['conversation'][i-1]
        if 'time' in msg and 'time' in prev_msg and msg['time'] < prev_msg['time']:
            print(f"Time inconsistency found: Message {i} ({msg['role']}) has time {msg['time']} which is earlier than previous message's time {prev_msg['time']}")

First conversation that fails time consistency check:

Analyzing time inconsistency issues:


In [89]:
a = '[\n    {\n        "role": "User",\n        "content": "What were the lessons I learned?",\n        "time": 1410.1\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 1431.0,\n        "start_time": 1411.0,\n        "end_time": 1431.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You sieve flour into a tray.",\n        "time": 1593.0,\n        "start_time": 1432.5,\n        "end_time": 1593.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 1618.5,\n        "start_time": 1594.0,\n        "end_time": 1618.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You add green peas to pot of oil.",\n        "time": 1638.0,\n        "start_time": 1620.5,\n        "end_time": 1638.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 1672.0,\n        "start_time": 1645.0,\n        "end_time": 1672.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You operate phone.",\n        "time": 1683.0,\n        "start_time": 1673.5,\n        "end_time": 1683.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 1697.0,\n        "start_time": 1683.5,\n        "end_time": 1697.0\n    },\n    {\n        "role": "User",\n        "content": "What was the outcome of my previous action?",\n        "time": 1703.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 1731.0,\n        "start_time": 1713.5,\n        "end_time": 1731.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You stir rice.",\n        "time": 1743.5,\n        "start_time": 1731.0,\n        "end_time": 1743.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You stir green peas.",\n        "time": 1750.5,\n        "start_time": 1744.5,\n        "end_time": 1750.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You scoop out part of sieved dough.",\n        "time": 1798.5,\n        "start_time": 1751.0,\n        "end_time": 1798.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 1855.5,\n        "start_time": 1799.0,\n        "end_time": 1855.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You add vegetables to green peas.",\n        "time": 1908.0,\n        "start_time": 1859.0,\n        "end_time": 1908.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 1930.5,\n        "start_time": 1911.0,\n        "end_time": 1930.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You scoop froth from surface of pot of rice.",\n        "time": 1936.0,\n        "start_time": 1931.0,\n        "end_time": 1936.0\n    },\n    {\n        "role": "User",\n        "content": "What were the key takeaways from what I did?",\n        "time": 2024.3\n    },\n    {\n        "role": "Assistant",\n        "content": "You stir rice.",\n        "time": 2154.5,\n        "start_time": 2138.0,\n        "end_time": 2154.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You stir vegetables.",\n        "time": 2179.5,\n        "start_time": 2155.0,\n        "end_time": 2179.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 2184.0,\n        "start_time": 2180.5,\n        "end_time": 2184.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You mix flour with water.",\n        "time": 2403.0,\n        "start_time": 2187.5,\n        "end_time": 2403.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 2410.0,\n        "start_time": 2403.0,\n        "end_time": 2410.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You remove rice from the fire.",\n        "time": 2435.5,\n        "start_time": 2411.0,\n        "end_time": 2435.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 2456.0,\n        "start_time": 2436.0,\n        "end_time": 2456.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You knead dough.",\n        "time": 2765.5,\n        "start_time": 2458.0,\n        "end_time": 2765.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 2780.5,\n        "start_time": 2766.5,\n        "end_time": 2780.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You sieve dough into tray.",\n        "time": 2833.5,\n        "start_time": 2780.5,\n        "end_time": 2833.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You stir vegetables.",\n        "time": 2864.0,\n        "start_time": 2846.0,\n        "end_time": 2864.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You wash hands.",\n        "time": 2935.5,\n        "start_time": 2865.0,\n        "end_time": 2935.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You add salt to vegetables.",\n        "time": 2993.0,\n        "start_time": 2958.0,\n        "end_time": 2993.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 3010.0,\n        "start_time": 2993.5,\n        "end_time": 3010.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You stir vegetables.",\n        "time": 3028.5,\n        "start_time": 3012.0,\n        "end_time": 3028.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You clean stove.",\n        "time": 3114.5,\n        "start_time": 3077.0,\n        "end_time": 3114.5\n    },\n    {\n        "role": "Assistant",\n        "content": "You add wood to fire.",\n        "time": 3155.0,\n        "start_time": 3115.5,\n        "end_time": 3155.0\n    },\n    {\n        "role": "Assistant",\n        "content": "You operate phone.",\n        "time": 3216.5,\n        "start_time": 3159.0,\n        "end_time": 3216.5\n    }\n]'
print(a)

[
    {
        "role": "User",
        "content": "What were the lessons I learned?",
        "time": 1410.1
    },
    {
        "role": "Assistant",
        "content": "You add wood to fire.",
        "time": 1431.0,
        "start_time": 1411.0,
        "end_time": 1431.0
    },
    {
        "role": "Assistant",
        "content": "You sieve flour into a tray.",
        "time": 1593.0,
        "start_time": 1432.5,
        "end_time": 1593.0
    },
    {
        "role": "Assistant",
        "content": "You add wood to fire.",
        "time": 1618.5,
        "start_time": 1594.0,
        "end_time": 1618.5
    },
    {
        "role": "Assistant",
        "content": "You add green peas to pot of oil.",
        "time": 1638.0,
        "start_time": 1620.5,
        "end_time": 1638.0
    },
    {
        "role": "Assistant",
        "content": "You add wood to fire.",
        "time": 1672.0,
        "start_time": 1645.0,
        "end_time": 1672.0
    },
    {
        "role": "Assist

In [113]:
import json

p = "/home/zhangyl/videollm-online/data/estp/annotation_tool/data_new_annotation/de8ea8a7-6bdb-4679-9b17-9dc6a79e45e7_c926f3f1-2ebe-4a02-9ae6-9cfcfbc21225/annotations.json"
a = json.load(open(p))
json.dump(a, open(p, 'w'), indent=4)

# gen final context qa bench

In [19]:
import json

# Load the existing data
object_data = json.load(open("/home/zhangyl/videollm-online/data/estp/annotation_tool/refine_annos_cqa_v2.json"))
task_data = json.load(open("/home/zhangyl/videollm-online/data/estp/annotation_train/0/goalstep_val_valid_samples_v3.json"))

final_data = object_data.copy()
for qa in task_data:
    video_uid = qa['video_uid']
    clip_uid = 'goal_step'
    if video_uid not in final_data:
        final_data[video_uid] = {}
    if clip_uid not in final_data[video_uid]:
        final_data[video_uid][clip_uid] = []
    qa['Task Type'] = 'Task Relative Context'
    final_data[video_uid][clip_uid].append(qa)

with open("/home/zhangyl/videollm-online/estp_bench_cq_v3.json", 'w') as f:
    json.dump(final_data, f, indent=4)

task2number = {
            "Object Relative Context": 0,
            "Task Relative Context": 0,
            "Irrelevant Context": 0,
        }
for k,v in final_data.items():
    for kk,vv in v.items():
        for vvv in vv:
            task2number[vvv['Task Type']] += 1

print(json.dumps(task2number, indent=4))


{
    "Object Relative Context": 219,
    "Task Relative Context": 211,
    "Irrelevant Context": 0
}
